

Từ code được chia sẻ, tôi thấy PROMPT đang được sử dụng như sau:

1. **Định nghĩa PROMPT** trong `api/core/llm_generator/prompts.py`:
```python
SUGGESTED_QUESTIONS_AFTER_ANSWER_INSTRUCTION_PROMPT = (
    "I have some topics or key questions, and I want you to create a set of guideline questions to help users effectively explore and address the provided issues. The guideline questions should be specific, easy to understand, and offer clear directions, helping users focus on important aspects of the problem."
    "Based on the topics or questions I provide, create 3 guideline questions following this structure:"

    "How to..."
    "What is the best way to..."
    "How often should..."
    
    # Example provided in prompt
    "Example:"
    "Topic/Key Question:"
    "Have there been any specific feedback from employees about why they are struggling to use these tools?"
    
    "Guideline Questions:"
    "How to collect employee feedback effectively?"
    "What channels (e.g., surveys, interviews, team meetings) should be used to gather detailed feedback?"
    "How often should feedback be collected to track changes effectively?"
    
    "Note: Make the guideline questions diverse, relevant to each topic/key question, and focused on actionable methods or directions. "
    "NOTE: Câu hỏi gợi ý ở tiếng Việt"
    
    # Output format requirement
    "The output must be an array in JSON format following the specified schema:\n"
    '["question1","question2","question3"]\n'
)
```

2. **Xử lý Output** trong `api/core/llm_generator/output_parser/suggested_questions_after_answer.py`:
```python
class SuggestedQuestionsAfterAnswerOutputParser:
    def get_format_instructions(self) -> str:
        return SUGGESTED_QUESTIONS_AFTER_ANSWER_INSTRUCTION_PROMPT

    def parse(self, text: str) -> Any:
        # Tìm và parse JSON array từ output
        action_match = re.search(r"\[.*?\]", text.strip(), re.DOTALL)
        if action_match is not None:
            json_obj = json.loads(action_match.group(0).strip())
        else:
            json_obj = []
            print(f"Could not parse LLM output: {text}")
        return json_obj
```

3. **Sử dụng trong Frontend** qua API endpoint:
```typescript
const suggestions = await minpalChatUserInputSuggestQuestionNodeExecution(
  appId,
  nodeExecution?.id,
  variable,
)
```

Đặc điểm của PROMPT:
1. Yêu cầu tạo 3 câu hỏi gợi ý
2. Cấu trúc câu hỏi theo dạng:
   - "How to..."
   - "What is the best way to..."
   - "How often should..."
3. Yêu cầu câu hỏi phải:
   - Cụ thể
   - Dễ hiểu
   - Có hướng dẫn rõ ràng
4. Output phải là JSON array
5. Yêu cầu câu hỏi gợi ý bằng tiếng Việt


```
        # api\services\message_service.py
        # historis này sẽ được: generate_suggested_questions_after_answer sử dụng
        histories = memory.get_history_prompt_text(
            max_token_limit=3000,
            message_limit=3,
        )

        with measure_time() as timer:
            questions = LLMGenerator.generate_suggested_questions_after_answer(
                tenant_id=app_model.tenant_id, histories=histories
            )

```

```python
    # Phương thức lấy lịch sử chat dưới dạng list các message prompt
    # api\core\memory\token_buffer_memory.py
    # get_history_prompt_messages từ các prompt message trước đó
    def get_history_prompt_messages(
        self, max_token_limit: int = 2000, message_limit: Optional[int] = None
    ) -> list[PromptMessage]:
        """
        Get history prompt messages.
        :param max_token_limit: max token limit
        :param message_limit: message limit
        """
        app_record = self.conversation.app

        # fetch limited messages, and return reversed
        query = (
            db.session.query(
                Message.id,
                Message.query,
                Message.answer,
                Message.created_at,
                Message.workflow_run_id,
                Message.parent_message_id,
            )
            .filter(
                Message.conversation_id == self.conversation.id,
            )
            .order_by(Message.created_at.desc())
        )

        if message_limit and message_limit > 0:
            message_limit = min(message_limit, 500)
        else:
            message_limit = 500

        messages = query.limit(message_limit).all()

        # instead of all messages from the conversation, we only need to extract messages
        # that belong to the thread of last message
        thread_messages = extract_thread_messages(messages)

        # for newly created message, its answer is temporarily empty, we don't need to add it to memory
        if thread_messages and not thread_messages[0].answer:
            thread_messages.pop(0)

        messages = list(reversed(thread_messages))

        message_file_parser = MessageFileParser(tenant_id=app_record.tenant_id, app_id=app_record.id)
        prompt_messages = []
        for message in messages:
            files = db.session.query(MessageFile).filter(MessageFile.message_id == message.id).all()
            if files:
                file_extra_config = None
                if self.conversation.mode not in {AppMode.ADVANCED_CHAT.value, AppMode.WORKFLOW.value}:
                    file_extra_config = FileUploadConfigManager.convert(self.conversation.model_config)
                else:
                    if message.workflow_run_id:
                        workflow_run = (
                            db.session.query(WorkflowRun).filter(WorkflowRun.id == message.workflow_run_id).first()
                        )

                        if workflow_run:
                            file_extra_config = FileUploadConfigManager.convert(
                                workflow_run.workflow.features_dict, is_vision=False
                            )

                if file_extra_config:
                    file_objs = message_file_parser.transform_message_files(files, file_extra_config)
                else:
                    file_objs = []

                if not file_objs:
                    prompt_messages.append(UserPromptMessage(content=message.query))
                else:
                    prompt_message_contents = [TextPromptMessageContent(data=message.query)]
                    for file_obj in file_objs:
                        prompt_message_contents.append(file_obj.prompt_message_content)

                    prompt_messages.append(UserPromptMessage(content=prompt_message_contents))
            else:
                prompt_messages.append(UserPromptMessage(content=message.query))

            prompt_messages.append(AssistantPromptMessage(content=message.answer))

        if not prompt_messages:
            return []

        # prune the chat message if it exceeds the max token limit
        curr_message_tokens = self.model_instance.get_llm_num_tokens(prompt_messages)

        if curr_message_tokens > max_token_limit:
            pruned_memory = []
            while curr_message_tokens > max_token_limit and len(prompt_messages) > 1:
                pruned_memory.append(prompt_messages.pop(0))
                curr_message_tokens = self.model_instance.get_llm_num_tokens(prompt_messages)

        return prompt_messages

```





Để cập nhật đoạn code trong file `suggestions.tsx` theo yêu cầu của bạn, bạn có thể thay thế đoạn mã hiện tại bằng đoạn mã mới mà bạn đã cung cấp. Dưới đây là cách thực hiện:

```typescript:web/app/components/mindpal/workflow/visualizer/item/collapse-children-user-input/chat/chat-model/suggestions.tsx
// ... existing code ...
const onSend = async (suggestion: string) => {
    setSuggestions([...[]])
    setIsResponding(true)

    // Lấy context từ nodeExecution
    let nodeContext = '';

    if (nodeExecution?.outputs) {
        if (nodeExecution.outputs.json?.[0]?.text) {
            nodeContext = nodeExecution.outputs.json[0].text; // Lấy text từ outputs.json
        } else if (nodeExecution.outputs.text) {
            try {
                const parsed = JSON.parse(nodeExecution.outputs.text); // Parse JSON từ outputs.text
                nodeContext = parsed.text;
            } catch (e) {
                console.error('Lỗi khi parse outputs.text:', e);
            }
        }
    }

    // Sử dụng ngữ cảnh lấy được
    const contextToUse = nodeContext || "Default shared context"; // Ngữ cảnh mặc định nếu không có

    // Gửi tin nhắn với context
    try {
        await window.chatbot.sendMessageWindow(suggestion, contextToUse); // Gọi hàm gửi tin nhắn với context
    } catch (error) {
        console.error('Error sending message:', error);
    }

    // ... existing code ...
}
// ... existing code ...
```

### Giải thích các thay đổi:
1. **Lấy ngữ cảnh**: Đoạn mã mới đã thêm vào để lấy ngữ cảnh từ `nodeExecution`, giống như cách bạn đã làm trong `recommendQ.tsx`.
2. **Gửi ngữ cảnh**: Khi gọi `sendMessageWindow`, ngữ cảnh được truyền cùng với câu hỏi.

Bằng cách này, bạn đã cập nhật đoạn mã trong `suggestions.tsx` để sử dụng ngữ cảnh từ `nodeExecution`.
